In [17]:
import os
import pynq
import numpy as np
# os.environ['EMCONFIG_PATH'] = os.environ['PWD']
ol=pynq.Overlay("krnl_matmulbertl_opt.xclbin")


In [128]:
from pynq import allocate
Nbanks=8
Nmat=1
Tsize=1024
Nvec=14

In [129]:
# low = 1 # -2^15
# high =  200# 2^15-1
low = -100
high = 100
source_v_np = np.random.randint(low, high, dtype=np.int8, size=(Tsize,Nvec))
source_v_np.shape

(1024, 14)

In [130]:
low = -100
high = 100
source_w_np = np.random.randint(low, high, dtype=np.int8, size=(Nmat*Tsize,Tsize))
source_w_np[100,13] = 2
source_w_np.shape


(1024, 1024)

In [131]:
source_w_split_np = []
for i in range(Nbanks):
    source_w_split_np.append(np.zeros((Nmat*Tsize,Tsize//Nbanks)))

In [132]:
source_w_split_np[0].shape

(1024, 128)

In [133]:
# save the split matrices of the reference values for the loading
source_w_split_np = np.hsplit(source_w_np, Nbanks)
source_w_split_np[0].shape

(1024, 128)

In [134]:
outbuf_np = np.zeros((Nvec, Tsize), dtype=np.int32)
outbuf_np.shape

(14, 1024)

In [135]:
outbuf_np = np.matmul(source_w_np, source_v_np, dtype=np.int32)
outbuf_np, outbuf_np.shape

(array([[ -90966,  -56078,   41803, ..., -113113,  -85729,   -8945],
        [-193090,  -22040,   -8048, ...,  105461,   62221,   66013],
        [  64896,  222697,  248032, ...,   99500,  -81101,   57323],
        ...,
        [-119448, -118695,   85969, ...,  -62000,  147851,  -66726],
        [  38542,  -27700,   76620, ...,  -96470,  133668,  -39413],
        [ 102315, -162639,   74915, ...,  -90539,  -70492,  -21505]],
       dtype=int32),
 (1024, 14))

In [136]:
source_w = [
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int8, target=ol.HBM0),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int8, target=ol.HBM4),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int8, target=ol.HBM8),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int8, target=ol.HBM12),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int8, target=ol.HBM16),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int8, target=ol.HBM20),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int8, target=ol.HBM24),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int8, target=ol.HBM26)]
 

In [137]:
source_w[0].shape, source_w_split_np[0].shape

((1024, 128), (1024, 128))

In [138]:
for i in range(len(source_w)):
#     source_w[i][:] = np.random.randint(-2^15, high=2^15-1, dtype=np.int8, size=(Nmat*Tsize//Nbanks,Tsize))
    source_w[i][:] = source_w_split_np[i]

In [139]:
source_w[0].shape


(1024, 128)

In [140]:
for i in range(8):
    source_w[i].sync_to_device()

In [141]:
source_v = pynq.allocate(shape=(Nvec,Tsize), dtype=np.int8, target=ol.HBM14)
print("source_v shape", source_v.shape)
# source_v[:] = np.random.randint(-2^15, high=2^15-1, dtype=np.int8, size=(Tsize,Nvec))
source_v[:] = source_v_np.T
print(type(source_v))
source_v.sync_to_device()
outbuf = pynq.allocate((Tsize*Nmat,Nvec), dtype=np.int32, target=ol.HBM14)
print("outbuf shape", outbuf.shape)

source_v shape (14, 1024)
<class 'pynq.buffer.PynqBuffer'>
outbuf shape (1024, 14)


In [142]:
%%timeit
ol.feeder_1.call(
	source_v,
	source_w[0],
	source_w[1],
	source_w[2],
	source_w[3],
	source_w[4],
	source_w[5],
	source_w[6],
	source_w[7],
    outbuf, 
    Nmat,
    Nvec, # seq length
    0)

outbuf.sync_from_device()
# nmat, nvec
# 1, 14: 1.16 ms ± 48.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
# 1, 128: 1.67 ms ± 19.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
# 3, 14 : 1.31 ms ± 17.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
# 3, 128: 3.32 ms ± 61.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
# 4, 128: 3.82 ms ± 46.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
# 
# 8, 128: 6.87 ms ± 77.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


1.16 ms ± 48.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [143]:
outbuf.shape

(1024, 14)

In [144]:
outbuf_np.shape

(1024, 14)

In [145]:
source_v

PynqBuffer([[-24,  70,  -3, ..., -59,  32,  51],
            [-77, -48, -17, ..., -42,  55, -28],
            [-69, -45,   2, ..., -72,  86, -11],
            ...,
            [ 79,  83,  90, ...,  54,  -2,  13],
            [-40, -39, -40, ..., -21, -37, -51],
            [ 98, -19,  43, ..., -54, -25, -42]], dtype=int8)

In [146]:
source_w[0]

PynqBuffer([[-64, -21, -77, ..., -75, -35,  31],
            [ 79,  17,  17, ...,  69,  41, -20],
            [ 89,  -3, -28, ..., -34,  32,  85],
            ...,
            [ 97, -76,  43, ...,  80, -97, -43],
            [-49, -70,  55, ..., -64,   2,  53],
            [ 74,   4,  88, ...,  35, -37,  55]], dtype=int8)

In [147]:
outbuf

PynqBuffer([[ -90966,  -56078,   41803, ..., -113113,  -85729,   -8945],
            [-193090,  -22040,   -8048, ...,  105461,   62221,   66013],
            [  64896,  222697,  248032, ...,   99500,  -81101,   57323],
            ...,
            [-119448, -118695,   85969, ...,  -62000,  147851,  -66726],
            [  38542,  -27700,   76620, ...,  -96470,  133668,  -39413],
            [ 102315, -162639,   74915, ...,  -90539,  -70492,  -21505]],
           dtype=int32)

In [148]:
outbuf_np

array([[ -90966,  -56078,   41803, ..., -113113,  -85729,   -8945],
       [-193090,  -22040,   -8048, ...,  105461,   62221,   66013],
       [  64896,  222697,  248032, ...,   99500,  -81101,   57323],
       ...,
       [-119448, -118695,   85969, ...,  -62000,  147851,  -66726],
       [  38542,  -27700,   76620, ...,  -96470,  133668,  -39413],
       [ 102315, -162639,   74915, ...,  -90539,  -70492,  -21505]],
      dtype=int32)

In [149]:
outbuf.any() == outbuf_np.any()

PynqBuffer(True)